In [1]:
import cdsapi
import requests
import os
import zipfile

In [2]:
start_year = 1992
end_year = 2022
year_range = [i for i in range(start_year, end_year + 1)]

In [3]:
#API Call to download past climate data
c = cdsapi.Client()
try:
    data = c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'format': 'grib',
        'product_type': 'monthly_averaged_reanalysis_by_hour_of_day',

        "variable": ['10m_u_component_of_wind', '10m_v_component_of_wind', 
                '2m_temperature',
                'total_cloud_cover', 
                'total_precipitation',
                '2m_dewpoint_temperature',
                ],
    "year": year_range,
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'area': [
            71, 
            -14, 
            31,
            46,
        ],  # North, West, South, East. 
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
    },)

    # Get the location of the file to download
    url = data.location

    # Download the file
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()

except requests.exceptions.HTTPError as errh:
    print ("HTTP Error:",errh)
except requests.exceptions.ConnectionError as errc:
    print ("Error Connecting:",errc)
except requests.exceptions.Timeout as errt:
    print ("Timeout Error:",errt)
except requests.exceptions.RequestException as err:
    print ("Something went wrong with the request:",err)

else:
    # If the request was successful, write the file 
    filename = 'past_climate_EU.grib'
    with open(filename, 'wb') as f:
        f.write(response.content)

    # Print the location where the file is saved
    print(f"File saved at: {os.path.abspath(filename)}")

2024-01-15 20:34:59,221 INFO Welcome to the CDS
2024-01-15 20:34:59,224 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2024-01-15 20:34:59,441 INFO Request is completed


File saved at: c:\Users\matthias\Documents\Projects\permaculture-climate\src\data\past_climate_EU.grib


In [5]:
# select the future years range
start_year_forecast = 2016
end_year_forecast = 2046
year_range_forecast = [str(i) for i in range(start_year_forecast, end_year_forecast + 1)]
#select the scenario you want to use for the projections.
#We reccomend ssp2_4_5, which is in our opinion a balanced options. 
# ssp1_1_9 ssp1_2_6 ssp4_3_4, ssp5_3_4os ssp2_4_5 ssp3_7_0 ssp5_8_5
#Note that if you choose the ssp4_6_0 parameter you also have to select another model in the API request as EC Earth3 doesnt provide that model. 

experiment = 'ssp2_4_5'
#variables that will be downloaded
pred_dataset_variables = ['near_surface_relative_humidity', 'near_surface_air_temperature', 'eastward_near_surface_wind', 
                     'northward_near_surface_wind', 'precipitation']


In [4]:
# Send API requests for the prediction data specified in dataset_variables. Each request returns a folder.zip
c = cdsapi.Client()

for variable in pred_dataset_variables:
        #create an if statement, as certain variables also include level, while others dont
    print(f'starting to download {variable}')
    data = c.retrieve(
        'projections-cmip6',
        {
            'format': 'zip',
            'temporal_resolution': 'monthly',
            'experiment': experiment,
            'variable': variable,
            'model': 'ec_earth3_cc',
            'area':[71, 
                    -14, 
                    31, 
                    46],  # North, West, South, East. 
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'year': year_range_forecast,

        },
        variable + '.zip')    
    #selectt the location of the file to download
    url = data.location
    # Download the file
    response = requests.get(url)
    # Check if the request was successful
    response.raise_for_status()

    # If the request was successful, write the file
    filename = variable + '.zip'
    with open(filename, 'wb') as f:
        f.write(response.content)

        # Print the location where the file is saved
        print(f"File saved at: {os.path.abspath(filename)}")
print('Downloads completed')

starting to download near_surface_relative_humidity


2024-01-08 13:47:56,721 INFO Welcome to the CDS
2024-01-08 13:47:56,723 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2024-01-08 13:47:56,847 INFO Request is queued
2024-01-08 13:47:57,905 INFO Request is running
2024-01-08 13:48:17,982 INFO Request is completed
2024-01-08 13:48:17,987 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data4/adaptor.esgf_wps.retrieve-1704718091.9406297-28390-17-85f76153-134e-4f97-b06d-1c9b5c0d4a71.zip to near_surface_relative_humidity.zip (6.5M)
2024-01-08 13:48:22,535 INFO Download rate 1.4M/s   
2024-01-08 13:48:26,449 INFO Welcome to the CDS
2024-01-08 13:48:26,453 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6


File saved at: c:\Users\matthias\Documents\Projects\permaculture-climate\src\data\near_surface_relative_humidity.zip
starting to download near_surface_air_temperature


2024-01-08 13:48:26,591 INFO Request is queued
2024-01-08 13:48:27,652 INFO Request is running
2024-01-08 13:48:47,833 INFO Request is completed
2024-01-08 13:48:47,835 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data4/adaptor.esgf_wps.retrieve-1704718121.8213434-3852-16-69ec0a09-0e28-4d54-8586-8b941d43b008.zip to near_surface_air_temperature.zip (5.9M)
2024-01-08 13:48:49,545 INFO Download rate 3.5M/s   
2024-01-08 13:48:56,273 INFO Welcome to the CDS
2024-01-08 13:48:56,276 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2024-01-08 13:48:56,350 INFO Request is queued


File saved at: c:\Users\matthias\Documents\Projects\permaculture-climate\src\data\near_surface_air_temperature.zip
starting to download eastward_near_surface_wind


...

File saved at: c:\Users\matthias\Documents\Projects\permaculture-climate\src\data\eastward_near_surface_wind.zip
starting to download northward_near_surface_wind


2024-01-08 13:49:27,881 INFO Request is queued
2024-01-08 13:49:28,962 INFO Request is running
2024-01-08 13:57:53,811 INFO Request is queued
2024-01-08 13:59:56,158 INFO Request is running
2024-01-08 14:16:15,749 INFO Request is completed
2024-01-08 14:16:15,752 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data4/adaptor.esgf_wps.retrieve-1704719731.6539087-30929-4-3f8ba805-0d9d-4e38-bade-44e97c6bc250.zip to northward_near_surface_wind.zip (7.1M)
2024-01-08 14:16:20,223 INFO Download rate 1.6M/s   
2024-01-08 14:16:24,682 INFO Welcome to the CDS
2024-01-08 14:16:24,685 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2024-01-08 14:16:24,802 INFO Request is queued


File saved at: c:\Users\matthias\Documents\Projects\permaculture-climate\src\data\northward_near_surface_wind.zip
starting to download precipitation


2024-01-08 14:16:25,860 INFO Request is running
2024-01-08 14:51:17,449 INFO Request is completed
2024-01-08 14:51:17,452 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data4/adaptor.esgf_wps.retrieve-1704721762.1241958-31366-4-1d53562d-06a7-421a-8032-62f684e5e463.zip to precipitation.zip (6.9M)
2024-01-08 14:51:21,791 INFO Download rate 1.6M/s   


File saved at: c:\Users\matthias\Documents\Projects\permaculture-climate\src\data\precipitation.zip
Downloads completed


In [6]:
extract_dir='prediction_data/'

# check if the folder already exist, if yes, delete files inside the folder
if os.path.exists(extract_dir):
    for file_name in os.listdir(extract_dir):
        file_path = os.path.join(extract_dir, file_name)
        if os.path.isfile(file_path):
            os.unlink(file_path)
# If not, create the folder
else:
    os.makedirs(extract_dir)

#extract and rename nc files from the zip folders into the new folder
for zip_file in os.listdir():
    #get zip folder paths
    if zip_file.endswith('.zip'):
        folder_name = os.path.splitext(zip_file)[0]       
        zip_file_path = os.path.join(os.getcwd(), zip_file)

        # Extract only the .nc file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            
            for file_info in zip_ref.infolist():
                # Extract only .nc files
                if file_info.filename.endswith('.nc'):
                    zip_ref.extract(file_info, extract_dir)               
                    
                    old_path = os.path.join(extract_dir, file_info.filename)
                    new_path = os.path.join(extract_dir, f'{folder_name}-{experiment}_{start_year_forecast}_{end_year_forecast}.nc')
                    os.rename(old_path, new_path)